## Building Blocks of DSPy

- **dspy.Signature**: define the input/output contract of DSPy module.
- **dspy.Module**: define the logic of interacting with LLMs.

## Advantages

- LM-agnostic programming
- Seamless productization
- Automatic program optimization

In [1]:
import dspy
from typing import Literal
from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())
dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"))

## Class-based Signature Example

In [2]:
class SlangEquivalence(dspy.Signature):
    """Find a Portuguese slang equivalance in English and classify it's formality level."""

    # inputs
    portuguese_slang: str = dspy.InputField()
    # outputs
    english_slang: str = dspy.OutputField()
    alternative_slangs: list[str] = dspy.OutputField(
        desc="More equivalent alternative slangs."
    )
    formality_level: Literal["L1", "L2", "L3", "L4", "L5"] = dspy.OutputField(
        desc="Slang formality level. Lesser the number more informal."
    )

## String-based Signature Example

In [3]:
slang_equivalance = dspy.make_signature(
    "portuguese_slang -> english_slang: str, alternative_slangs: list[str]"
)

## LM Interaction via Module Examples

In [4]:
predict = dspy.Predict(slang_equivalance)
output = predict(portuguese_slang="Boiar")

In [5]:
output

Prediction(
    english_slang='To be clueless or to not understand something.',
    alternative_slangs=['to zone out', 'to space out', 'to be lost', 'to be in the dark']
)

In [6]:
print(
    f"English equivalent slang: {output['english_slang']}\nAlternative slangs: {output.alternative_slangs}"
)

English equivalent slang: To be clueless or to not understand something.
Alternative slangs: ['to zone out', 'to space out', 'to be lost', 'to be in the dark']


In [7]:
cot = dspy.ChainOfThought(SlangEquivalence)
output = cot(portuguese_slang="quebrar o galho")

In [8]:
output

Prediction(
    reasoning='"Quebrar o galho" is a Portuguese slang that means to help someone out or to find a workaround for a problem. In English, a similar expression would be "to lend a hand" or "to help out." This phrase conveys a sense of informal assistance or support.',
    english_slang='lend a hand',
    alternative_slangs=['help out', 'give a hand', 'pitch in'],
    formality_level='L2'
)

In [9]:
dspy.inspect_history(n=1)





[2025-07-23T19:17:00.097117]

System message:

Your input fields are:
1. `portuguese_slang` (str):
Your output fields are:
1. `reasoning` (str): 
2. `english_slang` (str): 
3. `alternative_slangs` (list[str]): More equivalent alternative slangs.
4. `formality_level` (Literal['L1', 'L2', 'L3', 'L4', 'L5']): Slang formality level. Lesser the number more informal.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## portuguese_slang ## ]]
{portuguese_slang}

[[ ## reasoning ## ]]
{reasoning}

[[ ## english_slang ## ]]
{english_slang}

[[ ## alternative_slangs ## ]]
{alternative_slangs}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## formality_level ## ]]
{formality_level}        # note: the value you produce must exactly match (no extra characters) one of: L1; L2; L3; L4; L5

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Find 

## Changing the Adapter

In [12]:
dspy.settings.configure(lm=dspy.LM("openai/gpt-4o"))
dspy.configure(adapter=dspy.JSONAdapter())

In [13]:
cot = dspy.ChainOfThought(SlangEquivalence)
output = cot(portuguese_slang="quebrar o galho")

2025/07/23 19:19:48 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


In [15]:
output

Prediction(
    reasoning='"Quebrar o galho" is a Portuguese slang expression that means to help someone out or to do a favor, often in a situation where a quick or temporary solution is needed. The English equivalent would be "to help out" or "to lend a hand."',
    english_slang='help out',
    alternative_slangs=['lend a hand', 'do a favor', 'give a hand'],
    formality_level='L3'
)

In [14]:
dspy.inspect_history(n=1)





[2025-07-23T19:19:50.956426]

System message:

Your input fields are:
1. `portuguese_slang` (str):
Your output fields are:
1. `reasoning` (str): 
2. `english_slang` (str): 
3. `alternative_slangs` (list[str]): More equivalent alternative slangs.
4. `formality_level` (Literal['L1', 'L2', 'L3', 'L4', 'L5']): Slang formality level. Lesser the number more informal.
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## portuguese_slang ## ]]
{portuguese_slang}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "english_slang": "{english_slang}",
  "alternative_slangs": "{alternative_slangs}        # note: the value you produce must adhere to the JSON schema: {\"type\": \"array\", \"items\": {\"type\": \"string\"}}",
  "formality_level": "{formality_level}        # note: the value you produce must exactly match (no extra characters) one of: L1; L2; L3; L4

## Creating a Custom Module